# Computación Cuántica
## Búsqueda de Grover para problemas combinatorios
### Oscar Giovanny Duque Perdomo
### Julian Jacobo Londoño Suaza

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# importing Qiskit
from qiskit import Aer, IBMQ
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.compiler import transpile
from qiskit.tools.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor

In [ ]:
def black_box_u_f(circuit, f_in, f_out, aux, n, exactly_1_3_sat_formula):
    """Circuito que calcula la función de caja negra de f_in a f_out.

    A continuación, se construye un circuito que permita encontrar
    la solucion a un problema Exactamente-1 3-SAT
    """
    num_clauses = len(exactly_1_3_sat_formula)
    for (k, clause) in enumerate(exactly_1_3_sat_formula):
        
        # Bucle encargado de agregar las CNOT (XOR) y las compuertas X cuando el literal está negado
        for literal in clause:
            if literal > 0:
                circuit.cx(f_in[literal-1], aux[k])
            else:
                # Convierte el literal negativo a positivo y ubica la X y la CNOT
                circuit.x(f_in[-literal-1])
                circuit.cx(f_in[-literal-1], aux[k])
                
        # Invierte aux[k] si todos los literales son verdaderos
        circuit.ccx(f_in[0], f_in[1], aux[num_clauses])
        circuit.ccx(f_in[2], aux[num_clauses], aux[k])
        # Retorna al estado original, el qubit que se usa como auxiliar
        circuit.ccx(f_in[0], f_in[1], aux[num_clauses])
        
        # Retorna al estado original, los literales que se invertian al inicio,
        # con el objetivo de poder usarlos nuevamente
        for literal in clause:
            if literal < 0:
                circuit.x(f_in[-literal-1])
    
    # Condicional para graficar el grupo de CCNOT que sale al final de cada etapa,
    # el código se adaptó para que reciba hasta 5 clausulas
    if (num_clauses == 1):
        circuit.cx(aux[0], f_out[0])
    elif (num_clauses == 2):
        circuit.ccx(aux[0], aux[1], f_out[0])
    elif (num_clauses == 3):
        circuit.ccx(aux[0], aux[1], aux[num_clauses])
        circuit.ccx(aux[2], aux[num_clauses], f_out[0])
        circuit.ccx(aux[0], aux[1], aux[num_clauses])
    elif (num_clauses == 4):
        circuit.ccx(aux[0], aux[1], aux[num_clauses])
        circuit.ccx(aux[2], aux[num_clauses], f_out[1])
        circuit.ccx(aux[3], f_out[1], f_out[0])
        circuit.ccx(aux[2], aux[num_clauses], f_out[1])
        circuit.ccx(aux[0], aux[1], aux[num_clauses])
    elif (num_clauses == 5):
        circuit.ccx(aux[0], aux[1], aux[num_clauses])
        circuit.ccx(aux[2], aux[num_clauses], f_out[2])
        circuit.ccx(aux[3], f_out[2], f_out[1])
        circuit.ccx(aux[4], f_out[1], f_out[0])
        circuit.ccx(aux[3], f_out[2], f_out[1])
        circuit.ccx(aux[2], aux[num_clauses], f_out[2])
        circuit.ccx(aux[0], aux[1], aux[num_clauses])
    else:
        raise ValueError('We only allow at most 5 clauses')
    
    # Invertir los qubits auxiliares para asegurarse de que el estado sea consistente 
    # para futuras ejecuciones de esta rutina; mismo bucle que el anterior.
    for (k, clause) in enumerate(exactly_1_3_sat_formula):
        for literal in clause:
            if literal > 0:
                circuit.cx(f_in[literal-1], aux[k])
            else:
                circuit.x(f_in[-literal-1])
                circuit.cx(f_in[-literal-1], aux[k])
        circuit.ccx(f_in[0], f_in[1], aux[num_clauses])
        circuit.ccx(f_in[2], aux[num_clauses], aux[k])
        circuit.ccx(f_in[0], f_in[1], aux[num_clauses])
        for literal in clause:
            if literal < 0:
                circuit.x(f_in[-literal-1])
# -- end function

In [ ]:
def n_controlled_Z(circuit, controls, target):
    """ Implemente una compuerta Z con múltiples controles."""
    if (len(controls) > 2):
        raise ValueError('The controlled Z with more than 2 ' +
                         'controls is not implemented')
    elif (len(controls) == 1):
        circuit.h(target)
        circuit.cx(controls[0], target)
        circuit.h(target)
    elif (len(controls) == 2):
        circuit.h(target)
        circuit.ccx(controls[0], controls[1], target)
        circuit.h(target)
# -- end function

In [ ]:
def inversion_about_average(circuit, f_in, n):
    """Aplicar inversión sobre el paso promedio del algoritmo de Grover."""
    
    # Agregar compuertas Hadamards a todas los qubits de entrada
    for j in range(n):
        circuit.h(f_in[j])
    for j in range(n):
        circuit.x(f_in[j])
    n_controlled_Z(circuit, [f_in[j] for j in range(n-1)], f_in[n-1])
    for j in range(n):
        circuit.x(f_in[j])
    # Nuevamente se agregan compuertas Hadamards a todos los qubits de entrada
    for j in range(n):
        circuit.h(f_in[j])
# -- end function

In [ ]:
"""
Búsqueda de Grover implementada en Qiskit.

Este módulo contiene el código necesario para ejecutar la búsqueda de Grover en 3
qubits, tanto con un simulador como con una computación cuántica real dispositivo.
"""

def input_state(circuit, f_in, f_out, n):
    for j in range(n):
        circuit.h(f_in[j])
    circuit.x(f_out)
    circuit.h(f_out)
# -- end function

# n bits para el programa cuántico de la búsqueda de Grover.
n = 3

# Los literales se representan como enteros, positivos o negativos, 
# para indicar una variable booleana o su negación.
exactly_1_3_sat_formula = [[1, 2, -3], [-1, -2, -3], [-1, 2, 3]]

# Definir tres registros cuánticos: 'f_in' es el espacio de búsqueda 
# (entrada a la función f), 'f_out' es el bit utilizado para la salida
# de la función f, aux son los bits auxiliares utilizados por f para
# realizar su cálculo.
f_in = QuantumRegister(n)
f_out = QuantumRegister(3)
aux = QuantumRegister(len(exactly_1_3_sat_formula) + 1)

# Definir registro clásico para el resultado del algoritmo.
ans = ClassicalRegister(n)

# Definir circuito cuántico con los registros anteriores.
grover = QuantumCircuit()
grover.add_register(f_in)
grover.add_register(f_out)
grover.add_register(aux)
grover.add_register(ans)

input_state(grover, f_in, f_out, n)
T = 2

In [ ]:
for t in range(T):
    # Aplicar T iteraciones completas
    black_box_u_f(grover, f_in, f_out, aux, n, exactly_1_3_sat_formula)
    inversion_about_average(grover, f_in, n)

In [ ]:
# Medir el registro de salida en la base computacional
for j in range(n):
    grover.measure(f_in[j], ans[j])

# Ejecutar circuito
backend = Aer.get_backend('qasm_simulator')
job = execute([grover], backend=backend, shots=1000)
result = job.result()

In [ ]:
# Obtener resultados y trazar histograma
counts = result.get_counts(grover)
plot_histogram(counts)

In [ ]:
IBMQ.load_account()

In [ ]:
# Obtener la configuración ibmq_16_melbourne y el mapa de acoplamiento
provider = IBMQ.get_provider()
backend  = provider.get_backend('ibmq_16_melbourne')

In [ ]:
grover_circuit = grover
job = execute([grover_circuit], backend=backend, shots=1000, max_credits=10)
job_monitor(job, interval = 2)

In [ ]:
result = job.result()
answer = result.get_counts(grover_circuit)
plot_histogram(answer)

In [ ]:
# compilar el circuito para ibmq_16_rueschlikon
grover_compiled = transpile(grover, backend=backend, seed_transpiler=1, optimization_level=3)

print('gates = ', grover_compiled.count_ops())
print('depth = ', grover_compiled.depth())

In [ ]:
grover.draw(output='mpl', scale=0.5)